<a href="https://colab.research.google.com/github/roberttwomey/ml-art-code/blob/master/vgg_walkthrough/vgg19_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EMAR349 ML for the Arts - Twomey - Spring 2024 - [ml.roberttwomey.com](http://ml.roberttwomey.com)

In [ ]:
%load_ext autoreload
%autoreload 2

# Keras VGG19 Example for Image Classification
Import the keras modules and helper functions, as well as matplotlib.

Load the VGG19 model (from [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556), ICLR 2015).

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg19 import preprocess_input

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg19 import VGG19
model = VGG19()

In [ ]:
print(model.summary())

In [ ]:
# uncomment the following to plot a picture of the model

# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file='vgg.png')

the images we are going to analyze:

In [ ]:
!wget https://raw.githubusercontent.com/roberttwomey/ml-art-code/master/vgg_walkthrough/img/bus.jpg
!wget https://raw.githubusercontent.com/roberttwomey/ml-art-code/master/vgg_walkthrough/img/elephant.jpg

In [ ]:
fileList=['bus.jpg', 'elephant.jpg']

## Classify the Image

In [ ]:
for image_path in fileList:

    # load an image from file
    image = load_img(image_path, target_size=(224, 224))

    im2show = image

    # convert the image pixels to a numpy array
    image = img_to_array(image)

    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)

    # predict the probability across all output classes
    yhat = model.predict(image)

    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    # label = label[0][0]

    plt.figure(figsize=(4, 4))
    plt.axis("off")
    plt.imshow(im2show)
    plt.show()

    print("*******")
    # print matched labels
    for label in label[0]:
        # print the classification
        print('%s (%.2f%%)' % (label[1], label[2]*100))


# A Brief Tour of Layer Activations
We will step down through the layers and view a few activation maps. Define some helper functions to read and display the activations.

In [ ]:
from tensorflow.keras import Model

layer_names = ['input_1',
                'block1_conv1', 'block1_conv2', 'block1_pool',
                'block2_conv1', 'block2_conv2', 'block2_pool',
                'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_conv4', 'block3_pool',
                'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_conv4', 'block4_pool',
                'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_conv4', 'block5_pool',
                'flatten', 'fc1', 'fc2', 'predictions' ]

outputs = [model.get_layer(layer).output for layer in layer_names]

this_model = Model(inputs=model.input, outputs=outputs)

activations = this_model.predict(image)

## Step through the layers of the network

Run each of the below cells to display the feature maps ("activations") of each layer of the model.

In [ ]:
print(activations[0].shape)
plt.imshow(activations[0][0][:,:,0])

In [ ]:
len(activations)

Step through layers and view the activation maps. First, let's explore layer 0.

__Layer 0__: 224 x 224 pixels, 3 feature maps (channels).

This is the input image with three channels (RGB)

In [ ]:
activations[0].shape

show one of those images (R, G, or B). he last digit in this array `[0,:,:,0]` selects one of those 3 channels. (Currently __0__. Can be anything from __0-2__)

In [ ]:
plt.imshow(activations[0][0][:,:,0])


__Layer 1__: 224 x 224 pixels, 64 feature maps (channels)

In [ ]:
activations[1].shape

Select an individual feature map. The last digit in this array `[0,:,:,0]` selects one of those 64 channels. (Currently __0__. Can be anything from __0__-__63__) These are the first "features" extracted from the image.

In [ ]:
activations[1][0,:,:,0].shape

In [ ]:
plt.imshow(activations[0][0,:,:,2])

__Layer 2__: 224 x 224 pixels, 64 channels

In [ ]:
activations[2].shape

In [ ]:
activations[2][0,:,:,23].shape # 23 selects the 23rd feature map

In [ ]:
plt.imshow(activations[2][0,:,:,23])

__Layer 3__: 112 x 112 pixels

In [ ]:
activations[3].shape

In [ ]:
activations[3][0,:,:,47].shape

In [ ]:
plt.imshow(activations[3][0,:,:,47])

__Layer 6__: 56 x 56 pixels

In [ ]:
activations[6].shape

In [ ]:
activations[6][0,:,:,17].shape # 17th out of 128 feature maps

In [ ]:
plt.imshow(activations[6][0,:,:,17])

__Layer 11__: 28 x 28 pixels

In [ ]:
print(activations[11].shape)
print(activations[11][0,:,:,10].shape)
plt.imshow(activations[11][0,:,:,10]) # 10 out of 256 feature maps

__Layer 16__: 14 x 14 pixels

In [ ]:
print(activations[16].shape)
print(activations[16][0,:,:,1].shape)
plt.imshow(activations[16][0,:,:,1]) # 1 out of 512 feature maps

__Layer 25__: The last layer, 1 x 1000 wth softmax (categories).

First we can just print the values of the layer. This is hard to parse as a human reader.

In [ ]:
print(activations[25])

Reshaped to be 25 x 40 instead of a single long vector. The highest value (yellow spot) corresponds to the most likely predicted category.

In [ ]:
mtx=activations[25][0]
mtx.shape
mtx2=mtx.reshape(25,40)
fig=plt.imshow(mtx2)
plt.colorbar(fig)

In [ ]:
print("*******")
label = decode_predictions(activations[25])
# print matched labels
for label in label[0]:
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

# Activities
- Rerun some of the above cells, but to select a different channel/feature map. (change the last number in the array).
  - Can you discover any legible features?
- Upload your own image and run the image classification.
  - To add your own image, upload the file in the browser at right and add it to the end of the `filelist` array above. Re-run the classification step.
  - is the prediction correct?